## 20 Newsgroups

In [22]:
import matplotlib
import seaborn
import numpy as np
from sklearn import datasets
import tarfile
import re
import os
import pandas as pd

Getting category names, i.e. folder names of 20 newsgroups folder

In [4]:
path = "20_newsgroups"
for root, dirs, files in os.walk(path, topdown=False):
    newsgroups = np.array(dirs)
print newsgroups

['alt.atheism' 'comp.graphics' 'comp.os.ms-windows.misc'
 'comp.sys.ibm.pc.hardware' 'comp.sys.mac.hardware' 'comp.windows.x'
 'misc.forsale' 'rec.autos' 'rec.motorcycles' 'rec.sport.baseball'
 'rec.sport.hockey' 'sci.crypt' 'sci.electronics' 'sci.med' 'sci.space'
 'soc.religion.christian' 'talk.politics.guns' 'talk.politics.mideast'
 'talk.politics.misc' 'talk.religion.misc']


Reading files from first folder. Just use a for loop to read in from more folders.

In [173]:
path = "20_newsgroups" + "/" + newsgroups[0]
x = []
for root, dirs, files in os.walk(path):
    for name in files:
        f = open(path + "/" + name)
        x.append(f.read())
#print x[1]

Obtained metadatas From, Subject, Date, Organization, Sender from the headers of each file. Obtained Content from the body of each file and placed in the original dataframe.

In [213]:
original = pd.DataFrame()
original = original.assign(From = [])
original = original.assign(Subject = [])
original = original.assign(Date = [])
original = original.assign(Organization = [])
original = original.assign(Sender = [])
original = original.assign(Content = [])
original['From'] = original['From'].astype(str)
original['Subject'] = original['Subject'].astype(str)
original['Date'] = original['Date'].astype(str)
original['Organization'] = original['Organization'].astype(str)
original['Sender'] = original['Sender'].astype(str)
original['Content'] = original['Content'].astype(str)
print original.dtypes

for i in range(len(x)):
    data = x[i].split("\n\n", 1)
    email_content = ''
    meta = sorted(data[0].split("\n"))
    for k in meta:
        y = np.array(k.split(":", 1))
        if list(original).__contains__(y[0]):
            original.at[i, y[0]] = y[1]
    original.at[i, 'Content'] = data[1]

From            object
Subject         object
Date            object
Organization    object
Sender          object
Content         object
dtype: object
Archive-name: atheism/resources
Alt-atheism-archive-name: resources
Last-modified: 11 December 1992
Version: 1.0

                              Atheist Resources

                      Addresses of Atheist Organizations

                                     USA

FREEDOM FROM RELIGION FOUNDATION

Darwin fish bumper stickers and assorted other atheist paraphernalia are
available from the Freedom From Religion Foundation in the US.

Write to:  FFRF, P.O. Box 750, Madison, WI 53701.
Telephone: (608) 256-8900

EVOLUTION DESIGNS

Evolution Designs sell the "Darwin fish".  It's a fish symbol, like the ones
Christians stick on their cars, but with feet and the word "Darwin" written
inside.  The deluxe moulded 3D plastic fish is $4.95 postpaid in the US.

Write to:  Evolution Designs, 7119 Laurel Canyon #4, North Hollywood,
           CA 91605.


Extract email address from From and Sender column. From and Sender names were not considered as not much info can be gotten.  
Extra: extract domain, edu/country etc

In [210]:
original['Email_from'] = original.From.map(lambda x : re.search(r'[\w\.-]+@[\w\.-]+', x).group(0) if re.search(r'[\w\.-]+@[\w\.-]+', x) is not None else np.NaN)
print original['Email_from'].head()

original['Email_sender'] = original.Sender.map(lambda x : np.NaN if pd.isnull(x) else re.search(r'[\w\.-]+@[\w\.-]+', x).group(0) if re.search(r'[\w\.-]+@[\w\.-]+', x) is not None else np.NaN)
print original['Email_sender'].head()

0            mathew@mantis.co.uk
1            mathew@mantis.co.uk
2    I3150101@dbstu1.rz.tu-bs.de
3            mathew@mantis.co.uk
4           strom@Watson.Ibm.Com
Name: Email_from, dtype: object
0                         NaN
1                         NaN
2    postnntp@ibr.cs.tu-bs.de
3                         NaN
4                         NaN
Name: Email_sender, dtype: object


For Subject, remove Re: (maybe indicate that its a reply message), throw to content

For Date, extract month and year

## JunJie's Codes

In [174]:
with tarfile.open("C:/Users/admin/Desktop/training/20news-19997.tar.gz", 'r:gz') as tf:
    # get information (metadata) about all files in the tarball
    file_infos = [file_info for file_info in tf if file_info.isfile()]
    
    Extract_msg=[]    
    
    for i in xrange(1000):
        # print one of them; for example, the first one
        message = tf.extractfile(file_infos[i]).read()
        Extract_msg.append(message)

In [12]:
Compiled=[]

for message in Extract_msg:

    Content=message.split('\n\n')
    Email_content=''
    Meta_dict={}
   
    for index in xrange(len(Content)):
    
        if index == 0:
    
            Meta = sorted( Content[0].split('\n'))
            Meta_dict= { re.split(":\W" ,j)[0] : " ".join(re.split(":\W" ,j)[1:]) for j in Meta }

        elif len("".join(re.findall('[\-\-==]+',Content[index])))>80:
    
            break
        else:
            Email_content= Email_content + '\n'+ Content[index]    
        
    Meta_dict.update({'Content': Email_content})
    Compiled.append(Meta_dict)

In [13]:
Compiled[1]

{'Content': "\nIn article <1993Apr15.225657.17804@rambo.atlanta.dg.com>, wpr@atlanta.dg.com (Bill Rawlins) writes:\n|> \n|>        Since you have referred to the Messiah, I assume you are referring\n|>         to the New Testament.  Please detail your complaints or e-mail if\n|>         you don't want to post. \n|>        First-century Greek is well-known and\n|>         well-understood.  Have you considered Josephus, the Jewish Historian,\n|>         who also wrote of Jesus? \nThis can be now assumed kin of comic books.  There are many authors and artists\nwho write on, say, Superman.  This does not make the fictional character\nworthy of blind sycophantic worship.  Indeed, I too could write of Jesus.  Does\nthat make my writings sacred and me worthy of becoming a saint?  Somehow I doubt\nit.\n|>In addition, the four gospel accounts\n|>         are very much in harmony.  \n|> \n\n About a week ago a number of discrepancies were posted to this\nnewsgroup which noone, NOT ONE christian 

In [19]:
for i in range(len(Extract_msg[1].split("\n\n"))):
    print Extract_msg[1].split("\n\n")[i] + "\n\n"

Xref: cantaloupe.srv.cs.cmu.edu alt.atheism:53367 talk.religion.misc:83778 talk.origins:40952
Newsgroups: alt.atheism,talk.religion.misc,talk.origins
Path: cantaloupe.srv.cs.cmu.edu!das-news.harvard.edu!noc.near.net!uunet!pipex!uknet!warwick!nott-cs!mips.nott.ac.uk!eczcaw
From: eczcaw@mips.nott.ac.uk (C.Wainwright)
Subject: Re: Albert Sabin
Message-ID: <1993Apr16.133751.397@cs.nott.ac.uk>
Sender: news@cs.nott.ac.uk
Reply-To: eczcaw@mips.nott.ac.uk (C.Wainwright)
Organization: Nottingham University
References: <1993Mar29.231830.2055@rambo.atlanta.dg.com> <1993Apr7.073926.9874@engage.pko.dec.com> <1993Apr10.213547.17644@rambo.atlanta.dg.com> <1993Apr11.162936.18734@zeus.franklin.edu> <1993Apr15.225657.17804@rambo.atlanta.dg.com>
Date: Fri, 16 Apr 93 13:37:51 GMT
Lines: 37


In article <1993Apr15.225657.17804@rambo.atlanta.dg.com>, wpr@atlanta.dg.com (Bill Rawlins) writes:


|> 
|>        Since you have referred to the Messiah, I assume you are referring
|>         to the New Testament.  